In [1]:
import openai
import os
from openai import cli
import pandas as pd
import urllib.request
import json 
import urllib.request
from PIL import Image
import matplotlib.pyplot as plt
import requests
import time
import os
import matplotlib.image as mpimg
import io
import azure.ai.vision as visionsdk
import base64
import concurrent.futures
from azure.storage.blob import BlobServiceClient, ContentSettings

In [2]:
openai.api_key = "9dcb9b4900584019ab8f2c23eb8643d7"
openai.api_base = "https://chatgpttest45.openai.azure.com/"
openai.api_type = "azure"
openai.api_version = '2022-08-03-preview'

In [3]:
df= pd.read_csv('data.csv',index_col=[0])

In [4]:
df

,product_name,product_id,price,prompts,dalle_url,vector
0,Sparkling Sea Mermaid Doll,1,24.99,"Sparkling Sea Mermaid Doll, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[3.7792969, 3.7695312, 0.97753906, -1.7988281,..."
1,Mighty Monster Trucks Set,2,39.99,"Mighty Monster Trucks Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-2.0078125, -0.13623047, 0.14624023, 0.624023..."
2,Playful Pet Vet Clinic,3,34.99,"Playful Pet Vet Clinic, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[2.1542969, -2.9160156, -0.17004395, -1.238281..."
3,Rainbow Roller Skates,4,29.99,"Rainbow Roller Skates, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.5087891, 2.9960938, -2.234375, 1.5068359, 3..."
4,Build-A-Bot Robot Kit,5,54.99,"Build-A-Bot Robot Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.9951172, 1.3916016, -3.0, 1.1318359, 0.2440..."
...,...,...,...,...,...,...
82,Classic Pick-Up Sticks Game,95,12.99,"Classic Pick-Up Sticks Game, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.2539062, -0.71240234, 0.062805176, 1.160156..."
83,Miniature Tennis Set,97,19.99,"Miniature Tennis Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[0.41577148, -1.9677734, -0.8305664, -1.500976..."
84,My First Sewing Kit,98,24.99,"My First Sewing Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-4.7539062, 0.6020508, 1.8261719, 2.1640625, ..."
85,Foam Boomerang Set,99,16.99,"Foam Boomerang Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.0820312, -0.16967773, 0.89746094, -0.216308..."


In [12]:
def create_image_path(product_name):
    path = os.path.join(r'G:\VSCode\Personal_work\Vectorized_Search\app\image', f'{product_name}.jpeg')
    return path if os.path.isfile(path) else None

df['image_path'] = df['product_name'].apply(create_image_path)


In [13]:
df

,product_name,product_id,price,prompts,dalle_url,vector,image_path
0,Sparkling Sea Mermaid Doll,1,24.99,"Sparkling Sea Mermaid Doll, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[3.7792969, 3.7695312, 0.97753906, -1.7988281,...",G:\VSCode\Personal_work\Vectorized_Search\app\...
1,Mighty Monster Trucks Set,2,39.99,"Mighty Monster Trucks Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-2.0078125, -0.13623047, 0.14624023, 0.624023...",G:\VSCode\Personal_work\Vectorized_Search\app\...
2,Playful Pet Vet Clinic,3,34.99,"Playful Pet Vet Clinic, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[2.1542969, -2.9160156, -0.17004395, -1.238281...",G:\VSCode\Personal_work\Vectorized_Search\app\...
3,Rainbow Roller Skates,4,29.99,"Rainbow Roller Skates, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.5087891, 2.9960938, -2.234375, 1.5068359, 3...",G:\VSCode\Personal_work\Vectorized_Search\app\...
4,Build-A-Bot Robot Kit,5,54.99,"Build-A-Bot Robot Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.9951172, 1.3916016, -3.0, 1.1318359, 0.2440...",G:\VSCode\Personal_work\Vectorized_Search\app\...
...,...,...,...,...,...,...,...
82,Classic Pick-Up Sticks Game,95,12.99,"Classic Pick-Up Sticks Game, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.2539062, -0.71240234, 0.062805176, 1.160156...",G:\VSCode\Personal_work\Vectorized_Search\app\...
83,Miniature Tennis Set,97,19.99,"Miniature Tennis Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[0.41577148, -1.9677734, -0.8305664, -1.500976...",G:\VSCode\Personal_work\Vectorized_Search\app\...
84,My First Sewing Kit,98,24.99,"My First Sewing Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-4.7539062, 0.6020508, 1.8261719, 2.1640625, ...",G:\VSCode\Personal_work\Vectorized_Search\app\...
85,Foam Boomerang Set,99,16.99,"Foam Boomerang Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.0820312, -0.16967773, 0.89746094, -0.216308...",G:\VSCode\Personal_work\Vectorized_Search\app\...


In [15]:
df.to_csv('toy_data.csv')

In [39]:
import pandas as pd
df=pd.read_csv('full_data.csv',index_col=[0])

In [40]:
df

,product_name,product_id,price,prompts,dalle_url,vector,image_path,url
0,Sparkling Sea Mermaid Doll,1,24.99,"Sparkling Sea Mermaid Doll, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[3.7792969, 3.7695312, 0.97753906, -1.7988281,...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://live.staticflickr.com/65535/5297869297...
1,Mighty Monster Trucks Set,2,39.99,"Mighty Monster Trucks Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-2.0078125, -0.13623047, 0.14624023, 0.624023...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://live.staticflickr.com/65535/5297966701...
2,Playful Pet Vet Clinic,3,34.99,"Playful Pet Vet Clinic, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[2.1542969, -2.9160156, -0.17004395, -1.238281...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://live.staticflickr.com/65535/5297974853...
3,Rainbow Roller Skates,4,29.99,"Rainbow Roller Skates, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.5087891, 2.9960938, -2.234375, 1.5068359, 3...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://live.staticflickr.com/65535/5297974845...
4,Build-A-Bot Robot Kit,5,54.99,"Build-A-Bot Robot Kit, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.9951172, 1.3916016, -3.0, 1.1318359, 0.2440...",G:\VSCode\Personal_work\Vectorized_Search\app\...,https://live.staticflickr.com/65535/5297929273...
...,...,...,...,...,...,...,...,...
291,Zipping Funfair Play Set,288,39.99,"Zipping Funfair Play Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[1.8613281, 1.7744141, -1.8144531, 1.6083984, ...",NaN,NaN
292,Zigzagging Slot Car Set,290,49.99,"Zigzagging Slot Car Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-4.2304688, -1.6992188, -0.15686035, 2.228515...",NaN,NaN
293,Zany Zebra Play Set,292,24.99,"Zany Zebra Play Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-1.4677734, -4.7265625, -2.8125, 0.9370117, 4...",NaN,NaN
294,Zany Zipline Adventure Set,299,69.99,"Zany Zipline Adventure Set, kid's toy",https://dalleproduse.blob.core.windows.net/pri...,"[-2.2695312, -2.296875, 1.9746094, 2.5195312, ...",NaN,NaN


In [1]:
import flickrapi
import json

api_key = u'e08c541e809548ba4af46c81d462fe71'
api_secret = u'a61bdbf582176951'

flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# replace 'user_id' with your user ID
photos = flickr.people.getPhotos(user_id='198541622@N02',per_page=500)['photos']

titles=[]
urls=[]
for photo in photos['photo']:
    title = photo['title']
    server_id = photo['server']
    photo_id = photo['id']
    secret = photo['secret']
    size_suffix = 'b'  # replace with desired size suffix ('s', 'm', 'l', 'h', etc.)
    url = f"https://live.staticflickr.com/{server_id}/{photo_id}_{secret}_{size_suffix}.jpg"
    titles.append(title)
    urls.append(url)
    print(title, url)


U_mask https://live.staticflickr.com/65535/52987563543_a5c471eed6_b.jpg
U_mask https://live.staticflickr.com/65535/52986497762_66496d04a6_b.jpg
Leather Jacket https://live.staticflickr.com/65535/52986497892_40d6af301d_b.jpg
Dress https://live.staticflickr.com/65535/52986498092_28b4ccd382_b.jpg
Zooming Rocket Launch Set https://live.staticflickr.com/65535/52980257765_22f9d0a000_b.jpg
Zooming Race Car Set https://live.staticflickr.com/65535/52980257760_8f4a69eef6_b.jpg
Zookeeper Play Set https://live.staticflickr.com/65535/52980257770_1661776100_b.jpg
Zoo Train Play Set https://live.staticflickr.com/65535/52979276452_ef084424f2_b.jpg
Zoo Animals Play Set https://live.staticflickr.com/65535/52980338538_d99b85be43_b.jpg
Zombie Survival Play Set https://live.staticflickr.com/65535/52979276522_45f56459f4_b.jpg
Zodiac Star Sign Play Set https://live.staticflickr.com/65535/52980028314_a4aa645711_b.jpg
Zippy Zookeeper Adventure Set https://live.staticflickr.com/65535/52980257870_90402a940d_b.jp

In [52]:
len(titles)

297

In [53]:
# Convert your two lists to a dictionary
title_url_dict = dict(zip(titles, urls))

# Now create a new column 'url' in df
df['url'] = df['product_name'].map(title_url_dict)

In [54]:
df['url']

0      https://live.staticflickr.com/65535/5297869297...
1      https://live.staticflickr.com/65535/5297966701...
2      https://live.staticflickr.com/65535/5297974853...
3      https://live.staticflickr.com/65535/5297974845...
4      https://live.staticflickr.com/65535/5297929273...
                             ...                        
291    https://live.staticflickr.com/65535/5298002841...
292    https://live.staticflickr.com/65535/5297927665...
293    https://live.staticflickr.com/65535/5298033885...
294    https://live.staticflickr.com/65535/5297927685...
295    https://live.staticflickr.com/65535/5298025787...
Name: url, Length: 296, dtype: object

In [55]:
df.to_csv('toy_data.csv')